<a href="https://colab.research.google.com/github/jeonghyun-kim-jake/protowork/blob/main/solution/YOLOv8/20230407_YOLOv8_%ED%95%99%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [YOLOv8](https://github.com/ultralytics/ultralytics)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

! ln -s "/content/drive/MyDrive/Colab Notebooks/Datas/"  "/content/"
! ls -al "/content/Datas"

Mounted at /content/drive
lrwxrwxrwx 1 root root 45 Apr 10 12:44 /content/Datas -> '/content/drive/MyDrive/Colab Notebooks/Datas/'


In [2]:
%cd /content
! git clone https://github.com/jeonghyun-kim-jake/ultralytics_panning.git

/content
Cloning into 'ultralytics_panning'...
remote: Enumerating objects: 5239, done.
remote: Total 5239 (delta 0), reused 0 (delta 0), pack-reused 5239
Receiving objects: 100% (5239/5239), 2.77 MiB | 29.58 MiB/s, done.
Resolving deltas: 100% (3704/3704), done.


In [ ]:
%cd /content/ultralytics_panning
!git pull --rebase

In [3]:
#take a look at the kind of GPU we have
! nvidia-smi

Mon Apr 10 12:45:11 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
%cd /content
# install based on custom git project source code
! pip3 install -e ultralytics_panning

# install official ultralytics
# ! pip3 install ultralytics

/content
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/ultralytics_panning
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 19.7 MB/s eta 0:00:00
  Running setup.py develop for ultralytics


In [ ]:
# test yolo installation
! yolo task=detect mode=predict model=yolov8n.pt conf=0.25 source='https://media.roboflow.com/notebooks/examples/dog.jpeg'

In [6]:
%cd /content/ultralytics_panning
!cp "/content/drive/MyDrive/Colab Notebooks/scripts/yolov8_custom_data.yaml" ultralytics/datasets/

/content/ultralytics_panning


In [5]:

from pathlib import Path

class Project:
  def __init__(self, epochs, projectName="autoPanning", basePath="/content/drive/MyDrive/Colab Notebooks/Results"):
    self.project_name = getCurrentTimeStr(projectName)+"_epoch"+str(epochs)
    self.base_path = Path(basePath)
    self.save_dir= self.base_path / Path(self.project_name);
    self.best_model_path= self.base_path / self.project_name / "weights" / "best.pt";
    self.result_dir= self.save_dir / "results"

    print("Configured project", self.project_name)
    print("save_dir :", self.save_dir)
      

def getCurrentTimeStr(prefix="", format="%Y%m%d_%H%M_%S"):
   import datetime
   tmp = datetime.datetime.now().strftime(format)
   if len(prefix)>0:
     tmp = prefix+"_"+tmp
   return tmp


def deleteDirectory(path):
   import shutil
   try:
       shutil.rmtree(path)
       print(f"Folder '{path}' deleted successfully!")
   except OSError as e:
       print(f"Error deleting folder '{path}': {e}")



In [7]:
# parameter
# epochs = 2000
epochs = 5
save_period = 10
#######################

proj = Project(epochs)

print("Project will be saved on ", proj.save_dir)

from ultralytics import YOLO

# Load a model
model = YOLO('yolov8n.yaml').load('yolov8n.pt')  # build from YAML and transfer weights
# Train the model
model.train(data='yolov8_custom_data.yaml', epochs=epochs, imgsz=640, batch=25,
            device="0", save_period=save_period, project="autoPanning",single_cls=True, 
            name=proj.project_name, save_json=True, visualize=True, 
            save_txt=True, save_conf=True, 
            save_dir=proj.save_dir, patience=0)#, show=True, save_hybrid=True )

# Load best model from result
predict_model = YOLO(proj.best_model_path)
# Export onnx model file
predict_model.export(format='onnx')

# Inference with test images
results  = predict_model.predict(
   source='/content/Datas/AutoPanning/datasets/darknet/220405/test',
   conf=0.25,
   save_conf=True,
   save_txt=True,
   save_dir=proj.result_dir,
   visualize=True,
   save=True,
   save_crop=True,
   boxes=True,
   verbose=False
)

Configured project autoPanning_20230410_1251_04_epoch5
save_dir : /content/drive/MyDrive/Colab Notebooks/Results/autoPanning_20230410_1251_04_epoch5
Project will be saved on  /content/drive/MyDrive/Colab Notebooks/Results/autoPanning_20230410_1251_04_epoch5



                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.Conv                  [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.Conv                  [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.C2f                   [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.Conv                  [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.C2f                   [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.Conv                  [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.C2f                   [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics.nn.modules.Conv                  [128

save_dir is configured as  /content/ultralytics_panning/runs/detect/predict


AMP: checks passed ✅
optimizer: SGD(lr=0.01) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005859375000000001), 63 bias
train: Scanning /content/drive/MyDrive/Colab Notebooks/Datas/AutoPanning/datasets/darknet/220405/train.cache... 87 images, 0 backgrounds, 1 corrupt: 100%|██████████| 87/87 [00:00<?, ?it/s]
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))
val: Scanning /content/drive/MyDrive/Colab Notebooks/Datas/AutoPanning/datasets/darknet/220405/valid.cache... 7 images, 0 backgrounds, 1 corrupt: 100%|██████████| 7/7 [00:00<?, ?it/s]
Plotting labels to /content/drive/MyDrive/Colab Notebooks/Results/autoPanning_20230410_1251_04_epoch5/labels.jpg... 
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to /content/drive/MyDrive/Colab Notebooks/Results/autoPanning_20230410_1251_04_epoch5
Starting training for 5 epochs...

      Epoch    G

============= Diagnostic Run torch.onnx.export version 2.0.0+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

save_dir is configured as  /content/drive/MyDrive/Colab Notebooks/Results/autoPanning_20230410_1251_04_epoch5/results


visualize feature not yet supported
visualize feature not yet supported
visualize feature not yet supported
visualize feature not yet supported
visualize feature not yet supported
visualize feature not yet supported
visualize feature not yet supported
visualize feature not yet supported
visualize feature not yet supported
visualize feature not yet supported
visualize feature not yet supported
visualize feature not yet supported
visualize feature not yet supported
visualize feature not yet supported
visualize feature not yet supported
visualize feature not yet supported
visualize feature not yet supported
visualize feature not yet supported
visualize feature not yet supported
visualize feature not yet supported
visualize feature not yet supported
visualize feature not yet supported
visualize feature not yet supported
visualize feature not yet supported
visualize feature not yet supported
visualize feature not yet supported
visualize feature not yet supported
visualize feature not yet su

In [ ]:
# load tensorboard on colab
# https://amitness.com/2020/06/google-colaboratory-tips/
%tensorflow_version 2.x
%load_ext tensorboard
%tensorboard --logdir logs

In [ ]:
# ex) inference with trained model file
! yolo task=detect mode=predict model="/content/drive/MyDrive/Colab Notebooks/Results/autoPanning_20230407_1732_28_epoch2000/weights/best.pt" \
conf=0.25 source='/content/Datas/AutoPanning/datasets/darknet/220405/test' save=True project="inference_result"

In [ ]:
# ex) export onnx model
! yolo export model="/content/drive/MyDrive/Colab Notebooks/Results/autoPanning_20230407_1732_28_epoch2000/weights/best.pt" \
imgsz=640 format=onnx

In [ ]:
%cd /content/ultralytics_panning

from ultralytics import YOLO
from google.colab.patches import cv2_imshow

save_dir="/content/drive/MyDrive/Colab Notebooks/Results/autoPanning_20230407_1732_28_epoch2000"
model_path=save_dir+"/weights/best.pt";
result_dir=save_dir+"/results"
deleteDirectory(result_dir)

predict_model = YOLO(model_path)
results  = predict_model.predict(
   source='/content/Datas/AutoPanning/datasets/darknet/220405/test',
   conf=0.25,
   save_conf=True,
   save_txt=True,
   save_dir=result_dir,
   visualize=True,
   save=True,
   save_crop=True,
   boxes=True,
   verbose=False
)

ix = 0
for result in results:
    ix+=1
    print("[v]Result: ", ix)
    # boxes = result.boxes  # Boxes object for bbox outputs
    # masks = result.masks  # Masks object for segmentation masks outputs
    # probs = result.probs  # Class probabilities for classification outputs
    # print("Results: ", boxes, ", masks: ",masks, ", probs: ",probs)
    res_plotted = result.plot()
    cv2_imshow(res_plotted)

In [ ]:
# Run JupyterLab
!pip install jupyterlab pyngrok -q

# Run jupyterlab in the background
!nohup jupyter lab --ip=0.0.0.0 &

# Get ngrok URL mapped to port 8888
from pyngrok import ngrok
print(ngrok.connect(8888))